# 1. Modulos

In [ ]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 2. Rutas 

In [ ]:
# Ruta Entrada
R_Ms_ADRES_EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0019052025.TXT"
R_Ms_ADRES_EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-2\EPS025MS0019052025.TXT"
R_NC_NEG = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\NS\NS Negado\2025\NSEPS02516052025.NEG"
R_NC_Anterior = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\05_Mayo\16\NSEPS02516052025.txt"
# Ruta Salida
R_Salida = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\05_Mayo\22"

In [ ]:
Fecha = "22/05/2025"

# 3. Carga Dataframes

In [ ]:
new_columns = ["AFL_ID", "ENT_ID_ADRES", "TPS_IDN_ID_CF", "HST_IDN_NUMERO_IDENTIFICACION_CF", "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO", "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "AFL_PAIS_NACIMIENTO", "AFL_MUNICIPIO_NACIMIENTO", "AFL_NACIONALIDAD", "AFL_SEXO_IDENTIFICACION", "AFL_DISCAPACIDAD", "TPS_AFL_ID", "TPS_PRN_ID", "TPS_GRP_PBL_ID", "TPS_NVL_SSB_ID", "NUMEROFICHASISBEN", "TPS_CND_BNF_ID", "DPR_ID", "MNC_ID", "ZNS_ID", "AFL_FECHA_AFILIACION_SGSSS", "AFC_FECHA_INICIO", "NUMERO CONTRATO", "FECHADE INICIO DEL CONTRATO", "CNT_AFL_TPS_GRP_PBL_ID", "CNT_AFL_TPS_PRT_ETN_ID", "TPS_MDL_SBS_ID", "TPS_EST_AFL_ID", "CND_AFL_FECHA_INICIO", "CND_AFL_FECHA_INICIO", "GRP_FML_COTIZANTE_ID", "PORTABILIDAD", "COD_IPS_P", "MTDLG_G_P", "SUB_SISBEN_IV", "MARCASISBENIV+MARCASISBENIII", "CRUCE_BDEX_RNEC"]

Df_EPS025 = pd.read_csv(R_Ms_ADRES_EPS025, sep=',', header=None, dtype=str, encoding='ANSI')
Df_EPS025.columns = new_columns

Df_EPSC25 = pd.read_csv(R_Ms_ADRES_EPSC25, sep=',', header=None, dtype=str, encoding='ANSI')
Df_EPSC25.columns = new_columns

In [ ]:
# Dataframe NS anterior
new_columns = ["NUM_SOLICITUD_NOVEDAD", "ENT_ID", "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO", "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "DPR_ID", "MNC_ID", "NOVEDAD", "FECHA_NOVEDAD", "COD_1_NOVEDAD", "COD_2_NOVEDAD", "COD_3_NOVEDAD", "COD_4_NOVEDAD", "COD_5_NOVEDAD", "COD_6_NOVEDAD", "COD_7_NOVEDAD"]
Df_NS_Anterior = pd.read_csv(R_NC_Anterior, sep=',', header=None, dtype=str, encoding='ANSI')
Df_NS_Anterior.columns = new_columns
print("Número de registros en Df_S3:", Df_NS_Anterior.shape[0])

# Dataframe NS Glosado
new_columns.append("Glosa")
Df_NC_NEG = pd.read_csv(R_NC_NEG, sep=',', header=None, dtype=str, encoding='ANSI')
Df_NC_NEG.columns = new_columns
print("Número de registros en Df_S3:", Df_NC_NEG.shape[0])

# Agregar columna "Enviar" con un valor inicial vacío
Df_NC_NEG['Enviar'] = ''

# 4. Limpier datos

In [ ]:
# 1. Concatenar uno debajo del otro
DF_ADRES = pd.concat(
    [Df_EPS025, Df_EPSC25],
    ignore_index=True,   # reindexa de 0…n-1
    sort=False           # evita warnings si el orden de columnas coincide
)

# 2. (Opcional) borrar los DataFrames originales para liberar memoria
del Df_EPS025, Df_EPSC25

# 1) Tu merge original (trae ENT_ID_ADRES y TPS_EST_AFL_ID_from_adres de ADRES)
cols_transfer = ["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "ENT_ID_ADRES", "TPS_EST_AFL_ID"]
df_transfer  = DF_ADRES[cols_transfer].drop_duplicates()
Df_NC_NEG = Df_NC_NEG.merge(
    df_transfer,
    on=["TPS_IDN_ID","HST_IDN_NUMERO_IDENTIFICACION"],
    how="left"
).rename(columns={"TPS_EST_AFL_ID":"TPS_EST_AFL_ID_from_adres"})

# 2) Preparamos un mini-dataframe sólo con las filas N01 que sí sí cruzaron con ADRES
mapa_n01 = (
    Df_NC_NEG.loc[Df_NC_NEG["NOVEDAD"] == "N01", 
                  ["COD_1_NOVEDAD", "COD_2_NOVEDAD", "ENT_ID_ADRES", "TPS_EST_AFL_ID_from_adres"]]
    .copy()  # Asegurarse de trabajar con una copia del DataFrame
    .rename(columns={
        "COD_1_NOVEDAD": "TPS_IDN_ID",
        "COD_2_NOVEDAD": "HST_IDN_NUMERO_IDENTIFICACION",
        "ENT_ID_ADRES": "ENT_ID_from_self",
        "TPS_EST_AFL_ID_from_adres": "TPS_EST_AFL_ID_from_self"
    })
    .drop_duplicates(subset=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"])
)

# 3) Merge “secundario” contra esas N01 para rellenar vacíos
Df_NC_NEG = Df_NC_NEG.merge(
    mapa_n01,
    on=["TPS_IDN_ID","HST_IDN_NUMERO_IDENTIFICACION"],
    how="left"
)

# 4) Para los que quedaron con ENT_ID_ADRES NaN, y son evoluciones (p.ej. N02, N03, …),
#    rellenamos con el valor traído de la fila N01
mask = Df_NC_NEG["ENT_ID_ADRES"].isna() & Df_NC_NEG["NOVEDAD"].str.startswith("N0")
Df_NC_NEG.loc[mask, "ENT_ID_ADRES"]                   = Df_NC_NEG.loc[mask, "ENT_ID_from_self"]
Df_NC_NEG.loc[mask, "TPS_EST_AFL_ID_from_adres"] = Df_NC_NEG.loc[mask, "TPS_EST_AFL_ID_from_self"]

# 5) Limpiamos las columnas auxiliares
Df_NC_NEG.drop(columns=["ENT_ID_from_self","TPS_EST_AFL_ID_from_self"], inplace=True)


In [ ]:
print("Número de registros en Df_NC_NEG:", Df_NC_NEG.shape[0])

# 1. Definimos la máscara con la lógica (cond1 AND cond2)  OR  cond3
mask = (
    (Df_NC_NEG["ENT_ID_ADRES"] == "EPSC25")
)
# 2. Extraemos los registros a enviar
DF_EPSC25 = Df_NC_NEG.loc[mask].copy()

# 4. Definimos la máscara con la lógica (cond1 AND cond2)  OR  cond3
mask = (
    Df_NC_NEG["ENT_ID_ADRES"].isna()                 # NaN
    | (Df_NC_NEG["ENT_ID_ADRES"].astype(str).str.strip() == "")  # "" o "   "
)
# 5. Extraemos los registros a enviar
DF_No_Enviar = Df_NC_NEG.loc[mask].copy()

# 3. Eliminamos esos mismos registros del DataFrame original
Df_NC_NEG = Df_NC_NEG.loc[~mask].copy()

print("Número de registros en DF_EPSC25:", DF_EPSC25.shape[0])
print("Número de registros en DF_No_Enviar:", DF_No_Enviar.shape[0])
print("Número de registros en Df_NC_NEG:", Df_NC_NEG.shape[0])

In [ ]:
# Imprimir los valores únicos
print(Df_NC_NEG['NOVEDAD'].unique())
print(Df_NC_NEG['NOVEDAD'].value_counts())

In [ ]:
Df_NC_NEG